In [ ]:
import numpy as np
import math
import time
import scipy.stats as se

In [ ]:
S = 100
T = 1
r = 0.07
sigma = 0.2
Nsim = 50000
Nsteps = 250
K = 100

dt = T/Nsteps

drift = (r - (sigma**2)/2)*dt
a = sigma*np.sqrt(dt)
x = np.random.normal(0,1,(Nsim,Nsteps))

Smat = np.zeros((Nsim,Nsteps))
Smat[:,0]+=S
for i in range(1,Nsteps):
  Smat[:,i]+=Smat[:,i-1]*np.exp(drift+a*x[:,i])

q = Smat[:,-1]-K
for i in range(len(q)):
  if q[i]<0:
    q[i]=0
  else:
    q[i]=q[i]

p = K - Smat[:,-1]
for i in range(len(p)):
  if p[i]<0:
    p[i]=0
  else:
    p[i]=p[i]

payoff_call = np.mean(q)
payoff_put = np.mean(p)

payoff_call
payoff_put

call = payoff_call * np.exp(-r*T)
put = payoff_put * np.exp(-r*T)

print(call)
print(put)

In [ ]:
class OptionPricing:

  def __init__(self,S0,E,T,rf,sigma,iterations):
    self.S0 = S0
    self.E = E
    self.T = T
    self.rf = rf
    self.sigma = sigma
    self.iterations = iterations
  
  def call_option_simulation(self):

    option_data = np.zeros([self.iterations, 2])
    rand = np.random.normal(0, 1, [1,self.iterations])
    stock_price = self.S0 * np.exp(self.T * (self.rf - 0.5 * self.sigma**2)+self.sigma*np.sqrt(self.T)*rand)
    option_data[:,1] = stock_price - self.E
    average = np.sum(np.amax(option_data, axis=1)) / float(self.iterations)
    return np.exp(-1.0 * self.rf*self.T)*average

  def put_option_simulation(self):
    
    option_data = np.zeros([self.iterations, 2])
    rand = np.random.normal(0, 1, [1,self.iterations])
    stock_price = self.S0 * np.exp(self.T * (self.rf - 0.5 * self.sigma**2)+self.sigma*np.sqrt(self.T)*rand)
    option_data[:,1] = self.E - stock_price
    average = np.sum(np.amax(option_data, axis=1)) / float(self.iterations)
    return np.exp(-1.0 * self.rf*self.T)*average

if __name__ == "__main__":

  S0 = 100
  E = 100
  T = 1
  rf = 0.05
  sigma = 0.2
  iterations = 100000

  model = OptionPricing(S0, E, T, rf, sigma, iterations)
  print("Call option price with MonteCarlo approach: ", model.call_option_simulation())
  print("Put option price with MonteCarlo approach: ", model.put_option_simulation())


In [ ]:
def coe(rfr, b, mrp):
  return rfr + b * mrp

def wacc(cd, tax, debt, capital, rfr, b, mrp, equity):
  return cd * (debt/capital) * (1 - tax) + coe(rfr, b, mrp) * (equity/capital)

def d1(s, k, r, vol, t):
  return (np.log(s/k) + (r+(vol**2)/2)*t) / (vol * np.sqrt(t))

def d2(s, k, r, vol, t):
  return d1(s, k, r, vol, t) - vol*np.sqrt(t)

def callval(s, k, r, vol, t):
  return s * se.norm.cdf(d1(s, k, r, vol, t)) - k * se.norm.cdf(d2(s, k, r, vol, t))*np.exp(-r * t)

def putval(s, k, r, vol, t):
  return k * se.norm.cdf(-d2(s, k, r, vol, t)) * np.exp(-r * t) - s* se.norm.cdf(-d1(s, k, r, vol, t))

print(callval(100, 100, 0.07, 0.2, 1))
print(putval(100, 100, 0.07, 0.2, 1))